# Kurzy ČNB

Toto demo natáhne do notebooku a databáze kurzy ČNB za posledních 30 dnů.


## Inicializovat databázi

Nejprve načteme podporu pro SQL Server. Zároveň i NuGet balíček pro práci se SQL Serverem z C#, protože pokud ho přidáme později, restartuje se aplikace a přijdeme o proměnné.

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"
#r "nuget:System.Data.SqlClient"

Nyní definujeme připojení `kurzyCnb`. Do connection stringu se nezadává databáze, protože v tuto chvíli nemusí existovat.

In [ ]:
#!connect mssql --kernel-name kurzyCnb "SERVER=.\SqlExpress;TRUSTED_CONNECTION=yes"


Pokud již existuje, smažeme databázi KurzyCNB. Poté vytvoříme novou prázdnou.

In [ ]:
#!sql-kurzyCnb

-- Drop current database
IF DB_ID('KurzyCNB') IS NOT NULL BEGIN
    USE master
    ALTER DATABASE KurzyCNB SET SINGLE_USER WITH ROLLBACK IMMEDIATE
    DROP DATABASE KurzyCNB
END

-- Create a new database
CREATE DATABASE KurzyCNB
GO

-- Use the newly created database
USE KurzyCNB


Vytvoříme databázovou strukturu (tabulku `ExchangeRates`).

In [ ]:
#!sql-kurzyCnb

CREATE TABLE [DailyExchangeRates] (
    [Date]  datetime        not null,
    [USD]   decimal(10, 3)  not null,
    [EUR]   decimal(10, 3)  not null,
    [GBP]   decimal(10, 3)  not null,
)

## Zkusit API 

Vytvoříme instanci WebClienta a zkusíme stáhnout dnešní kurzy z ČNB, abychom se podívali na data.

In [ ]:
var apiUrl = "https://www.cnb.cz/cs/financni_trhy/devizovy_trh/kurzy_devizoveho_trhu/denni_kurz.txt?date={0:dd\\.MM\\.yyyy}";
var wc = new System.Net.Http.HttpClient();
var csv = await wc.GetStringAsync(string.Format(apiUrl, DateTime.Today));

csv

Napíšeme jednoduchý parser, který data rozloží na řádky a poté na pole.

In [ ]:
var csvLines = csv.Split('\n').Skip(2);
var csvData = csvLines.Select(x => x.Split('|'));

csvData

Zkusíme si vyhledat hodnotu kurzu EUR a proparsovat ho jako desetinné číslo. Použijeme českou kulturu, kvůli desetinné čárce.


In [ ]:
var czechCulture = System.Globalization.CultureInfo.GetCultureInfo("cs-CZ");
var value = decimal.Parse(csvData.First(x => x[3].Equals("EUR"))[4], czechCulture);

value

## Načíst z API data za posledních 30 dnů

Vytvoříme strukturu `DailyExchangeRate`, která bude držet informace o kurzech z jednoho dne.

In [ ]:
struct DailyExchangeRate {
    public DateTime Date;
    public decimal Usd;
    public decimal Eur;
    public decimal Gbp;
}

Ve smyčce načteme data za posledních 30 dnů.

In [ ]:
var exchangeRates = new List<DailyExchangeRate>();
var days = 30;

for(var i = -days + 1; i <=0; i++) {
    var date = DateTime.Today.AddDays(i);
    var csv = await wc.GetStringAsync(string.Format(apiUrl, date));
    var csvLines = csv.Split('\n').Skip(2);
    var csvData = csvLines.Select(x => x.Split('|'));
    exchangeRates.Add(new DailyExchangeRate {
        Date = date, 
        Usd = decimal.Parse(csvData.First(x => x[3].Equals("USD"))[4], czechCulture),
        Eur = decimal.Parse(csvData.First(x => x[3].Equals("EUR"))[4], czechCulture),
        Gbp = decimal.Parse(csvData.First(x => x[3].Equals("GBP"))[4], czechCulture),
    });
}

exchangeRates

## Uložit tato data do databáze

Připravíme si databázi a parametrický dotaz:

In [ ]:
using System.Data;
using System.Data.SqlClient;

var db = new SqlConnection("SERVER=.\\SqlExpress;TRUSTED_CONNECTION=yes;DATABASE=KurzyCNB");
// Prepare SQL Command
var cmd = db.CreateCommand();
cmd.CommandText = "INSERT INTO DailyExchangeRates (Date, USD, EUR, GBP) VALUES (@Datum, @USD, @EUR, @GBP)";
cmd.Parameters.Add(new SqlParameter("@Datum", SqlDbType.DateTime));
cmd.Parameters.Add(new SqlParameter("@USD", SqlDbType.Decimal));
cmd.Parameters.Add(new SqlParameter("@EUR", SqlDbType.Decimal));
cmd.Parameters.Add(new SqlParameter("@GBP", SqlDbType.Decimal));

Postupně nahrneme data do databáze.

In [ ]:
db.Open();

foreach(var dayRate in exchangeRates){
    cmd.Parameters["@Datum"].Value = dayRate.Date;
    cmd.Parameters["@USD"].Value = dayRate.Usd;
    cmd.Parameters["@EUR"].Value = dayRate.Eur;
    cmd.Parameters["@GBP"].Value = dayRate.Gbp;
    await cmd.ExecuteNonQueryAsync();
}

db.Close();

Ověříme že data jsou skutečně v databázi

In [ ]:
#!sql-kurzyCnb

SELECT * FROM DailyExchangeRates

## Použít Entity Framework

Definujeme připojení do DB a tentokrát vytvoříme i DB Context:

In [ ]:
#!connect mssql --create-dbcontext --kernel-name kurzyCnbDc "SERVER=.\SqlExpress;TRUSTED_CONNECTION=yes;DATABASE=KurzyCNB"


Nyní se můžeme dotazovat do databázové struktury pomocí EF/LINQ:

In [ ]:
var kurzy = kurzyCnbDc.DailyExchangeRates.OrderByDescending(x => x.Date);

kurzy

In [ ]:
var nejlevnejsiEur = kurzyCnbDc.DailyExchangeRates.OrderBy( x => x.Eur).Select(x => new {x.Date, x.Eur}).First();

nejlevnejsiEur